In [1]:
!pip -q install langchain_core langchain_chroma langchain langchain_groq

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
deepspeed 0.10.3 requires pydantic<2.0.0, but you have pydantic 2.11.1 which is incompatible.
gradient 2.0.6 requires attrs<=19, but you have attrs 23.1.0 which is incompatible.
gradient 2.0.6 requires PyYAML==5.*, but you have pyyaml 6.0.2 which is incompatible.
wandb 0.15.10 requires protobuf!=4.21.0,<5,>=3.19.0; python_version > "3.9" and sys_platform == "linux", but you have protobuf 5.29.4 which is incompatible.
tensorflow 2.15.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 5.29.4 which is incompatible.
tensorboard 2.15.1 requires protobuf<4.24,>=3.19.6, but you have protobuf 5.29.4 which is incompatible.


In [2]:
!pip -q install -U langchain-chroma

In [3]:
from langchain_core.documents import Document
documents=[
    Document(
        page_content="Dogs are great companions",
        metadata={"source":"mamal-pets-doc"}),
      Document(
        page_content="Cats are great companions",
        metadata={"source":"mamal-pets-cat-doc"})
    
]

In [4]:
documents

[Document(metadata={'source': 'mamal-pets-doc'}, page_content='Dogs are great companions'),
 Document(metadata={'source': 'mamal-pets-cat-doc'}, page_content='Cats are great companions')]

In [6]:
##vectorstores
from langchain_chroma import Chroma
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()
os.environ['HF_TOKEN']=os.getenv('HF_TOKEN')
os.environ['GROQ_API_KEY']=os.getenv('GROQ_API_KEY')
llm=ChatGroq(model="Llama3-8b-8192")

In [7]:
!pip -q install langchain_huggingface

In [9]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling%2Fconfig.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
vectorstore=Chroma.from_documents(documents, embeddings)

In [11]:
vectorstore

In [12]:
vectorstore.similarity_search("cat")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(id='76b10324-33a8-4018-95ff-9f5afc37ef4d', metadata={'source': 'mamal-pets-cat-doc'}, page_content='Cats are great companions'),
 Document(id='5950f30c-62a2-4501-934f-d7a4bf1f04a0', metadata={'source': 'mamal-pets-doc'}, page_content='Dogs are great companions')]

In [14]:
#Async query
await vectorstore.asimilarity_search("cat")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[Document(id='76b10324-33a8-4018-95ff-9f5afc37ef4d', metadata={'source': 'mamal-pets-cat-doc'}, page_content='Cats are great companions'),
 Document(id='5950f30c-62a2-4501-934f-d7a4bf1f04a0', metadata={'source': 'mamal-pets-doc'}, page_content='Dogs are great companions')]

In [16]:
vectorstore.similarity_search_with_score("dog")

Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


[(Document(id='5950f30c-62a2-4501-934f-d7a4bf1f04a0', metadata={'source': 'mamal-pets-doc'}, page_content='Dogs are great companions'),
  0.9212460517883301),
 (Document(id='76b10324-33a8-4018-95ff-9f5afc37ef4d', metadata={'source': 'mamal-pets-cat-doc'}, page_content='Cats are great companions'),
  1.3184987306594849)]

In [27]:
## Retrievers
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='76b10324-33a8-4018-95ff-9f5afc37ef4d', metadata={'source': 'mamal-pets-cat-doc'}, page_content='Cats are great companions')],
 [Document(id='5950f30c-62a2-4501-934f-d7a4bf1f04a0', metadata={'source': 'mamal-pets-doc'}, page_content='Dogs are great companions')]]

In [29]:
retriever=vectorstore.as_retriever(
search_type="similarity",
    search_kwargs={"k":1}
)


In [30]:
retriever.batch(['cat','dog'])

[[Document(id='76b10324-33a8-4018-95ff-9f5afc37ef4d', metadata={'source': 'mamal-pets-cat-doc'}, page_content='Cats are great companions')],
 [Document(id='5950f30c-62a2-4501-934f-d7a4bf1f04a0', metadata={'source': 'mamal-pets-doc'}, page_content='Dogs are great companions')]]

In [31]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

In [35]:
message=""" Answer this question using provided context
{question}
Context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([('human',message)])

rag_chain={'context':retriever,'question':RunnablePassthrough()} | prompt | llm

In [ ]:
response=rag_chain.invoke("Hey tell me about dogs?")